# Visualizing the newly created zarrs with Lexcube

In [18]:
import os
import xarray as xr
import lexcube as shittyLexcube

In [19]:
import sys
sys.executable

'/home/sc.uni-leipzig.de/ju554xqou/.conda/envs/shittyLexcube/bin/python'

In [24]:
zarr_base="/work/ju554xqou-rhonezarrs"
os.chdir(zarr_base)
zarrs=os.listdir()
testcube=xr.open_dataset(zarrs[0], chunks={}, engine="zarr")
#test.fft.sel(time='2020-07-04T15:50:30.049000000').plot()
test=testcube.fft[0:10000].T #[0:10,0:10,0:10]
test

<xarray.DataArray 'fft' (frequency: 101, channel: 2300, time: 10000)> Size: 9GB
dask.array<transpose, shape=(101, 2300, 10000), dtype=float32, chunksize=(101, 2300, 60), chunktype=numpy.ndarray>
Coordinates:
  * channel    (channel) int64 18kB 0 4 8 12 16 20 ... 9180 9184 9188 9192 9196
  * frequency  (frequency) float64 808B 0.0 1.0 2.0 3.0 ... 97.0 98.0 99.0 100.0
  * time       (time) datetime64[ns] 80kB 2020-07-04T15:50:30.049000 ... 2020...

In [12]:
import numpy as np
import fnmatch 

def extract_timestamp(filename):
    """
    Extracts the timestamp from a DAS-h5-file's filename.
    
    Args:
        filename (str): The filename to extract the timestamp from.
        
    Returns:
        str: The timestamp part of the filename.
    """
    # for the format is 'rhone1khz_UTC_yyyymmdd_hhmmss.ms.h5'
    timestamp_part = filename.split('_')[2] + filename.split('_')[3]
    return timestamp_part

def get_filenames(folder, base):
    """
    Collects the filenames in the data folder and sorts them by time.

    Args:
        folder (str): The folder to search for files.

    Returns:
        dict: A dictionary where keys are integers and values are filenames, sorted chronologically.
    """
    # Change to the folder directory
    folder_path = os.path.join(base, folder)
    os.chdir(folder_path)
    files=os.listdir()
    
    # filter files to only get those where the filename contains 'rhone2das' or 'rhone1khz'
    # Use fnmatch.filter to find matches for both patterns in a single pass
    pattern_matches = fnmatch.filter(files, '*rhone2das*') + fnmatch.filter(files, '*rhone1khz*')
    
    # Sort the files by timestamp
    sorted_files = sorted(pattern_matches, key=extract_timestamp)
    
    return  sorted_files 

base="/work/le837wmue-Rhone_download/DAS_2020"
os.chdir(base)
folder=os.listdir()[0]
os.chdir(folder)
time_res=0.1
freq_res=1
filenames = get_filenames(folder, base)
n_files=len(filenames)
file_length = 30 # Length of a single h5 file in seconds.
sample_freq = 1000 #Sampling frequency in Hz of the recorded data.
freq_max = 100 # maximum frequency cut off value for the analysis
n_samples = file_length*sample_freq
seg_length=1/freq_res #calculate window length corresponding to freq_res
hop = int(time_res*sample_freq)
seg_sample_len=int(seg_length*sample_freq)
n_segments_total=int(np.floor((n_files*n_samples-seg_sample_len)/hop))+1 # total amount of segments

dummy_file_path=os.path.join(base, folder, filenames[0])
dummy_xr = xr.open_dataset(dummy_file_path, engine='h5netcdf', backend_kwargs={'phony_dims': 'access'})
attr = dummy_xr['Acoustic'].attrs
start_time = np.datetime64(attr["ISO8601 Timestamp"], 'ns') # Get the start time of the first file
time_res_ms = time_res * 1000  # Convert time_res from seconds to milliseconds
time_coords = start_time + np.arange(n_segments_total) * np.timedelta64(int(time_res_ms), 'ms') 

/tmp/ipykernel_1405304/2207416225.py:62: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  start_time = np.datetime64(attr["ISO8601 Timestamp"], 'ns') # Get the start time of the first file


In [13]:
len(time_coords)

863991

In [14]:
np.arange(n_segments_total)

array([     0,      1,      2, ..., 863988, 863989, 863990])

In [15]:
testcube.assign_coords(time=time_coords)

ValueError: conflicting sizes for dimension 'time': length 863991 on 'time' and length 293691 on {'channel': 'channel', 'time': 'fft', 'frequency': 'fft'}

In [25]:
w = shittyLexcube.Cube3DWidget(test.compute().T)
w

Xarray input object does not have chunks. You can re-open with 'chunks={}' to enable dask for caching and progress reporting functionality - but may be overall slower for small data sets.


Cube3DWidget(api_metadata={'/api': {'status': 'ok', 'api_version': 5}, '/api/datasets': [{'id': 'default', 'sh…

In [22]:
w.show_sliders()

Sliders(children=(HBox(children=(IntRangeSlider(value=(0, 0), description='frequency:'),)), HBox(children=(Int…